<a href="https://colab.research.google.com/github/NarsinAshritha/ML_miniproject/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import f1_score,accuracy_score

df=pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
df=pd.DataFrame(df)
df


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
df['sentiment'].value_counts()
df['review'][0]



"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [3]:
import nltk
#to remove stopwords
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)
#to remove html tags
from bs4 import BeautifulSoup
def html_tag(text):
 soup=BeautifulSoup(text,"html.parser")
 new_text=soup.get_text()
 return new_text

html_tag("<html<h2> beauty </h2></html>")  



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


' beauty '

In [4]:
#expand contractions
!pip install contractions
import contractions
def cont(text):
  expand=contractions.fix(text)
  return expand
#to remove special characters
import re                             #regular expression library
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text
#cont("you can't do these aren't easy!")  
#remove_sp("!!reminder**!!!don't forget")


     |████████████████████████████████| 317kB 6.2MB/s 
     |████████████████████████████████| 245kB 35.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81704 sha256=8da70a065638c8b265fe22b79635621234902f5174f4fc88bcd6b96432ef912b
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [5]:
#to remove stopwords
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text
remove_stopwords("the, and,is,are stopwords and even not, no")  
  

', , , stopwords even not , no'

In [6]:
df.review=df.review.apply(lambda x:x.lower())
df.review=df.review.apply(html_tag)
df.review=df.review.apply(cont)
df.review=df.review.apply(remove_sp)
df.review=df.review.apply(remove_stopwords)
df.head()


,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter matteis love time money visually stunni...,positive


In [7]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()
df['compound score']=df['review'].apply(lambda x: vs.polarity_scores(x)['compound'])
df.head()



     |████████████████████████████████| 133kB 4.2MB/s 


,review,sentiment,compound score
0,one reviewers mentioned watching 1 oz episode ...,positive,-0.9905
1,wonderful little production filming technique ...,positive,0.9172
2,thought wonderful way spend time hot summer we...,positive,0.9251
3,basically family little boy jake thinks zombie...,negative,-0.9117
4,petter matteis love time money visually stunni...,positive,0.9754


In [8]:
df.shape

(50000, 3)

In [9]:
df['compound score'][0]

-0.9905

In [10]:
#model_sentiment -a new column for getting the type of statement (positive or negative or neutral) from the model built
df.loc[df['compound score'] <= -0.05, 'model_sentiment'] = 'negative statement' 
df.loc[df['compound score'] >=0.05, 'model_sentiment'] = 'positive statement'
df.loc[(df['compound score'] >-0.05) & (df['compound score']<0.05), 'model_sentiment'] = 'neutral statement'
df
 

,review,sentiment,compound score,model_sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive,-0.9905,negative statement
1,wonderful little production filming technique ...,positive,0.9172,positive statement
2,thought wonderful way spend time hot summer we...,positive,0.9251,positive statement
3,basically family little boy jake thinks zombie...,negative,-0.9117,negative statement
4,petter matteis love time money visually stunni...,positive,0.9754,positive statement
...,...,...,...,...
49995,thought movie right good job not creative orig...,positive,0.9618,positive statement
49996,bad plot bad dialogue bad acting idiotic direc...,negative,-0.9320,negative statement
49997,catholic taught parochial elementary schools n...,negative,-0.9651,negative statement
49998,I going disagree previous comment side maltin ...,negative,-0.8192,negative statement


In [11]:
df['model_sentiment'].value_counts()           

positive statement    33018
negative statement    16397
neutral statement       585
Name: model_sentiment, dtype: int64

In [12]:
df['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [13]:
#save the model using pickle library
import pickle
pickle.dump(df,open("text_analysis.p",'wb'))
model=pickle.load(open("text_analysis.p",'rb'))